In [0]:
#Priya Rajpurohit 2015073
#Sakshi Saini 2017092

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from zipfile import ZipFile
import os
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import pickle
import h5py
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive1 = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=10fFInHmGZ5PsbMTRb_7EuhJcGNHisjCi'
fluff, id = link.split('=')

downloaded = drive1.CreateFile({'id':id}) 
downloaded.GetContentFile('Segmented.zip')


In [0]:
with ZipFile('/content/Segmented.zip', 'r') as zipObj:
   zipObj.extractall()

In [0]:
path = '/content/Segmented/'
images=[]
lable = 1
labels = []
for r, d, f in os.walk(path):
    for folder in d:
        
        for r1, d1, f1 in os.walk(os.path.join(r, folder)):
          for file in f1:
            # print(file)
            images.append( cv2.imread(os.path.join(r1, file) ))
            labels.append(lable)

          lable = lable +1

In [0]:
images = np.array(images)

In [0]:
X_train, X_test, y_train, y_test = train_test_split( images[:10000], labels[:10000], test_size=0.3, random_state=42 , shuffle=True)

In [0]:
model = VGG16( weights='imagenet', include_top=False )

features = []
for image in X_train:

  image = cv2.resize(image , (224, 224) )
  image = preprocess_input(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

  feature = model.predict(image)
  features.append(feature)

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
featuresTest = []
for image in X_test:

  image = cv2.resize(image , (224, 224) )
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  feature = model.predict(image)
  featuresTest.append(feature)

In [0]:
features = np.array(features)
features = features.reshape(7000, 25088)

clf = RandomForestClassifier()
clf.fit(features,  y_train)

In [0]:
featuresTest = np.array(featuresTest)
featuresTest = featuresTest.reshape(3000, 25088)

# accuracy = clf.score(featuresTest,y_test)
# accuracy *= 100
# print(accuracy)

NameError: ignored

In [0]:
filename = 'rf_model_segmented.sav'
pickle.dump(clf, open(filename, 'wb'))

In [0]:
clfSVM = svm.SVC()
clfSVM.fit(features, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
accuracy = clfSVM.score(featuresTest,y_test)
accuracy *= 100
print(accuracy)

98.1


In [0]:
filename = 'svm_model_final.sav'
pickle.dump(clfSVM, open(filename, 'wb'))

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd 
y = pd.read_csv("/content/drive/My Drive/SML/labels.csv") 

In [0]:
### For Binary Classification
n = featuresTest.shape[0]

healthy = []
diseased = []

score = 0
for i in range(n):
  image = featuresTest[i].reshape(1, -1)
  pred = clfSVM.predict(image)
  actual = y_test[i]

  x1, boole = testActual(pred, actual)
  if(boole):
    score +=1

  if( x1 == 'healthy' ):
    healthy.append(image)
  else:
    diseased.append(image)
accuracy = score/n
print(accuracy)

In [0]:
def testActual(pred, actual):
  
  l1 = y['label_name'][pred[0] ]
  l2 = y['label_name'][actual]
  # print(l1,l2)

  if 'healthy' in l1:
    x1='healthy'
  else:
    x1='diseased'
  if 'healthy' in l2:
    x2='healthy'
  else:
    x2='diseased'

  if( x1 == x2 ):
    return (x1, True)

  return (x1, False)

In [0]:
### For Disease Classification
n = featuresTest.shape[0]

score = 0
for i in range(n):
  image = featuresTest[i].reshape(1, -1)
  pred = clfSVM.predict(image)
  actual = y_test[i]

  x1, boole = testActual(pred, actual)
  if(boole):
    score +=1

  if( x1 == 'healthy' ):
    healthy.append(image)
  else:
    diseased.append(image)
accuracy = score/n
print(accuracy)